#Data Wrangling

###Print Spark Version and Import Libraries

In [68]:
println("Spark Version : " + sc.version)

import org.apache.commons.io.IOUtils
import java.net.URL
import java.nio.charset.Charset
import com.databricks.spark.csv
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType,DoubleType,DateType,TimestampType};
import scala.util.matching.Regex
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql._
import org.apache.spark.sql.types.{StructType,StructField,StringType,DoubleType};
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Column
import org.apache.spark.sql.functions.{udf,col,concat,lit,when,not,avg,asc}



Spark Version : 1.6.0


### Add Boilerplate for loading data from Swift object store - REVAMP

In [31]:
def setConfig(credentials : scala.collection.mutable.HashMap[String, String]) = {
    val prefix = "fs.swift.service." + credentials("name") 
    var hconf = sc.getConf
    hconf.set(prefix + ".auth.url", credentials("auth_url")+"/v3/auth/tokens")
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials("project_id"))
    hconf.set(prefix + ".username", credentials("user_id"))
    hconf.set(prefix + ".password", credentials("password"))
    hconf.set(prefix + ".http.port", "8080")
    hconf.set(prefix + ".region", credentials("region"))
    hconf.set(prefix + ".public", "True")   
}

var credentials_1 = scala.collection.mutable.HashMap[String, String](
  "auth_url"->"https://identity.open.softlayer.com",
  "project"->"object_storage_0dc58f04_2c63_4761_9aed_ef4449bae858",
  "project_id"->"f4162fef734849528c38937ca26ba3a7",
  "region"->"dallas",
  "user_id"->"e4fc729439f044199998e68ef469e3c5",
  "domain_id"->"7527e49e459a4de4a41373a7e8fb444f",
  "domain_name"->"984499",
  "username"->"Admin_66474152b3dcb39af2943f3f79bb2618f028659f",
  "password"->"""pIv/Y^MOZ17^LQ{j""",
  "filename"->"scores_nba.test.dat",
  "container"->"notebooks",
  "tenantId"->"s69c-1c345cae431193-591ae43264f0"
)

credentials_1("name") = "spark"
setConfig(credentials_1)
//val nbafile = sc.textFile("swift://notebooks." + credentials_1("name") + "/" + credentials_1("filename"))

val nbafile  = "swift://notebooks.spark/scores_nba.test.dat"
val oddsfile = "swift://notebooks.spark/nbaodds_042516.xml"


### Use local filesystem to store small initial files from GitHub

In [10]:
import sys.process._

// Store the current working directory for this notebook in variable yourDir
val yourDir="pwd".!!.replaceAll("\n", "")

// Get the source data files from GitHub
"wget https://raw.githubusercontent.com/dustinvanstee/nba-rt-prediction/master/nbaodds_042516.xml".!
"wget https://raw.githubusercontent.com/dustinvanstee/nba-rt-prediction/master/scores_nba.test.dat".!

// Make sure the file is in the directory
"ls -lart".!

// Setup paths to data files for later use in notebooks
val nbafile  = yourDir+"/scores_nba.test.dat"
val oddsfile = yourDir+"/nbaodds_042516.xml"

// Print out a few lines from the files
Seq("head", "-n", "5", oddsfile).! 
Seq("head", "-n", "5", nbafile).! 



--2016-09-14 09:00:31--  https://raw.githubusercontent.com/dustinvanstee/nba-rt-prediction/master/nbaodds_042516.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21937 (21K) [text/plain]
Saving to: 'nbaodds_042516.xml.2'

     0K .......... .......... .                               100% 22.6M=0.001s

2016-09-14 09:00:31 (22.6 MB/s) - 'nbaodds_042516.xml.2' saved [21937/21937]

--2016-09-14 09:00:33--  https://raw.githubusercontent.com/dustinvanstee/nba-rt-prediction/master/scores_nba.test.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1301197 (1.2M) [text/plain]
Saving to: 'scores_nba.test.dat.1'

    

0

2016-04-05,15:06:16,Phoenix,0,Atlanta,0,(8:00 PM ET),48.0,400829044
2016-04-05,15:06:16,Chicago,0,Memphis,0,(8:00 PM ET),48.0,400829045
2016-04-05,15:06:16,Cleveland,0,Milwaukee,0,(8:00 PM ET),48.0,400829046
2016-04-05,15:06:16,Oklahoma City,0,Denver,0,(9:00 PM ET),48.0,400829047
2016-04-05,15:06:16,New Orleans,0,Philadelphia,0,(7:00 PM ET),48.0,400829041


In [1]:
import sys.process._
"df -h".!

Filesystem      Size  Used Avail Use% Mounted on
/dev/sda3       930G  119G  812G  13% /
devtmpfs        189G     0  189G   0% /dev
tmpfs           189G     0  189G   0% /dev/shm
tmpfs           189G  258M  189G   1% /run
tmpfs           189G     0  189G   0% /sys/fs/cgroup
/dev/sde1       3.6T   89M  3.4T   1% /disk4
/dev/sdh1       3.6T   89M  3.4T   1% /disk7
/dev/sdg1       3.6T   89M  3.4T   1% /disk6
/dev/sdj1       3.6T   89M  3.4T   1% /disk9
/dev/sdb1       3.6T  9.4G  3.4T   1% /disk1
/dev/sdi1       3.6T   89M  3.4T   1% /disk8
/dev/sdf1       3.6T   89M  3.4T   1% /disk5
/dev/sdd1       3.6T   89M  3.4T   1% /disk3
/dev/sdc1       3.6T   89M  3.4T   1% /disk2
/dev/sda1       253M  161M   93M  64% /boot
tmpfs            38G     0   38G   0% /run/user/0
/dev/fs01       246T   80T  167T  33% /gpfs/global_fs01


0

### Helpers ... Lookup Tables

In [11]:
val teamMap = Map(
  "Atlanta" -> "atl",
  "Boston"  -> "bos",
  "Brooklyn"  -> "bkn",
  "Charlotte"  -> "cha",
  "Chicago"  -> "chi",
  "Cleveland"  -> "cle",
  "Dallas"  -> "dal",
  "Denver"  -> "den",
  "Detroit"  -> "det",
  "Golden State"  -> "gst",
  "Houston"  -> "hou",
  "Indiana"  -> "ind",
  "LA Clippers"  -> "lac",
  "LA Lakers"  -> "lal",
  "Memphis"  -> "mem",
  "Miami"  -> "mia",
  "Milwaukee"  -> "mil",
  "Minnesota"  -> "min",
  "New Orleans"  -> "nor",
  "New York"  -> "nyk",
  "Oklahoma City"  -> "okc",
  "Orlando"  -> "orl",
  "Philadelphia"  -> "phi",
  "Phila."  -> "phi",
  "Phoenix"  -> "pho",
  "Portland"  -> "por",
  "Sacramento" -> "sac",
  "San Antonio"  -> "san",
  "Toronto"  -> "tor",
  "Utah"  -> "uta",
  "Washington"  -> "wsh")
  
val monthMap = Map(
    "Jan"-> "01",
    "Feb"-> "02",
    "Mar"-> "03",
    "Apr"-> "04",
    "May"-> "05",
    "Jun"-> "06",
    "Jul"-> "07",
    "Aug"-> "08",
    "Sep"-> "09",
    "Oct"-> "10",
    "Nov"-> "11",
    "Dec"-> "12"
)

### Inspect Score Data ( remove ??)

In [33]:
// TODO ## cat a file in swift ?? Prob not possible
println(nbafile)
import sys.process._
"head swift://notebooks.spark/scores_nba.test.dat".!
"df -h".!
"pwd".!
"touch /gpfs/global_fs01/sym_shared/YPProdSpark/user/s69c-1c345cae431193-591ae43264f0/notebook/notebooks/junk".!

swift://notebooks.spark/scores_nba.test.dat
head: cannot open 'swift://notebooks.spark/scores_nba.test.dat' for reading: No such file or directory
Filesystem      Size  Used Avail Use% Mounted on
/dev/sda3       930G  119G  812G  13% /
devtmpfs        189G     0  189G   0% /dev
tmpfs           189G     0  189G   0% /dev/shm
tmpfs           189G  226M  189G   1% /run
tmpfs           189G     0  189G   0% /sys/fs/cgroup
/dev/sde1       3.6T   89M  3.4T   1% /disk4
/dev/sdh1       3.6T   89M  3.4T   1% /disk7
/dev/sdg1       3.6T   89M  3.4T   1% /disk6
/dev/sdj1       3.6T   89M  3.4T   1% /disk9
/dev/sdb1       3.6T  9.4G  3.4T   1% /disk1
/dev/sdi1       3.6T   89M  3.4T   1% /disk8
/dev/sdf1       3.6T   89M  3.4T   1% /disk5
/dev/sdd1       3.6T   89M  3.4T   1% /disk3
/dev/sdc1       3.6T   89M  3.4T   1% /disk2
/dev/sda1       253M  161M   93M  64% /boot
tmpfs            38G     0   38G   0% /run/user/0
/dev/fs01       246T   81T  166T  33% /gpfs/global_fs01
/gpfs/global_fs01/sym_s

0

###Load In NBA Score Data Sets

In [12]:
// Since I dont have a header in the data set, I want to specify the column metadata
val sqlContext = new SQLContext(sc)
// Need this for my shorthand $ notation
import sqlContext.implicits._  

val customSchema = StructType(Array(
    StructField("dateOrig", DateType, true),
    StructField("ts", StringType, true),
    StructField("teamlonga", StringType, true),
    StructField("scorea", IntegerType, true),
    StructField("teamlongb", StringType, true),
    StructField("scoreb", IntegerType, true),
    StructField("time-string", StringType, true),
    StructField("timeleft", DoubleType, true),
    StructField("gameid", IntegerType, true)
    ))

// This line reads in the file and parses it with a CSV reader
// Use first line of all files as header.
// Automatically infer data types)
var rtscoresAndFinalDF = sqlContext.read.format("com.databricks.spark.csv").
    option("header", "false").
    option("inferSchema", "false").
    option("nullValue", "empty").
    option("dateFormat", "yyyy-MM-dd").
    option("mode","DROPMALFORMED").
    schema(customSchema).load(nbafile) 
    


### Inspect Historical Score Data

In [13]:
rtscoresAndFinalDF.cache()
rtscoresAndFinalDF.show(5)
rtscoresAndFinalDF.filter($"time-string".contains("FINAL")).show(5)
rtscoresAndFinalDF.filter(rtscoresAndFinalDF("time-string").contains("1ST")).show(5)

+----------+--------+-------------+------+------------+------+------------+--------+---------+
|  dateOrig|      ts|    teamlonga|scorea|   teamlongb|scoreb| time-string|timeleft|   gameid|
+----------+--------+-------------+------+------------+------+------------+--------+---------+
|2016-04-05|15:06:16|      Phoenix|     0|     Atlanta|     0|(8:00 PM ET)|    48.0|400829044|
|2016-04-05|15:06:16|      Chicago|     0|     Memphis|     0|(8:00 PM ET)|    48.0|400829045|
|2016-04-05|15:06:16|    Cleveland|     0|   Milwaukee|     0|(8:00 PM ET)|    48.0|400829046|
|2016-04-05|15:06:16|Oklahoma City|     0|      Denver|     0|(9:00 PM ET)|    48.0|400829047|
|2016-04-05|15:06:16|  New Orleans|     0|Philadelphia|     0|(7:00 PM ET)|    48.0|400829041|
+----------+--------+-------------+------+------------+------+------------+--------+---------+
only showing top 5 rows

+----------+--------+-----------+------+------------+------+-----------+--------+---------+
|  dateOrig|      ts|  teaml

### UDFs For Creating Extra Columns In Real Time Data Frame

In [36]:
////////////////////////////////////////////////////////////
// These were a couple custom UDF's I needed to cleanse the data 
// and also to add a few features based on a proprietary way of combining
// the score with the time left.

// Create new team name column.. do simple lookup conversion with a UDF
val mapper = teamin => teamMap(teamin)
val mapperudf = udf(mapper)


// Date Logic to adjust for games that finish on the day after ....
// This is due to not having a great key to join my tables ...
val datecrossregex = new Regex("^0[0-3]")
val dateadjust : ((String, String) => String) = (datein, tsin ) => {
    val datetest =  datecrossregex.findFirstIn(tsin)
    val dateary = datein.split("-")
    val rv = datetest match {
      case Some(s) => { 
         val day = "%02d".format(dateary(2).toInt -1)
         val newdate = dateary(0) + "-" + dateary(1) + "-" + day  
         newdate
      }
      case None => datein
    }
    rv.asInstanceOf[String]
}
val dateadjustudf = udf(dateadjust)

// UDFs to create some extra features ... this one is for an experiemental combination of Time left and Score difference.  
// Made this via intuition.  This can be extended to add other custom features
//val crossOverTime = 8
//val exponentScaler = 0.5
val scoredivtimeXform: ((Double,Double,Double,Double) => Double) = (sd:Double, tl:Double, co:Double, exp:Double) => {
    val scaler = 1 / Math.pow( (tl / co) + 0.01 , exp)
    sd * scaler
}
val scoredivtimeUdf = udf(scoredivtimeXform)



###Preproces the Real Time and Final Score Data .  Add some useful columns to the data set

In [37]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////
// Here I create some extra columns for later use.  


// Remove Overtime games from this analysis
rtscoresAndFinalDF = rtscoresAndFinalDF.filter(!$"time-string".like("%OT%"))

// Create short 3 character team names 
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("teama", mapperudf(col("teamlonga")))
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("teamb", mapperudf(col("teamlongb")))

// Add a score differential Column 
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("scorea-scoreb", $"scorea" - $"scoreb")

// Transform the Date.  This is for games that spanned multiple days and gave me a headache.  
// Games adjusted to the day they started on.
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("date",  dateadjustudf($"dateOrig",$"ts"))

// Create a Key for me to use to join with odds data later.  Key = date.teama.teamb
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("key", concat($"date",lit("."),$"teama",lit("."),$"teamb"))

###Separate The Real Time And Final Data From One Common Dataframe To Two Dataframes

In [38]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////
// Currently based on the way the data was sampled, both real time scores and final scores are written as seperate records to the same file. 
// I need to pull these apart, and then join the dataframes so that I have a real time score and features and know if the game was won or lost ....

///////////////////////////////////////////////////////////////////////////////////////////////////////////////
// Create Final Score DF
// Note a shortcut for repeating the dataframe within the filter is to use a $   df.filter(df("foo").contains ... is equiv to df.filter($"foo".contains)

var finalscoresDF = rtscoresAndFinalDF.filter($"time-string".like("%FINAL%"))

// Rename some columns so that join later doesnt have name overlaps
finalscoresDF = finalscoresDF.withColumnRenamed("scorea", "fscorea")
finalscoresDF = finalscoresDF.withColumnRenamed("scoreb", "fscoreb")

// Create final score difference
finalscoresDF = finalscoresDF.withColumn("fscorea-fscoreb", $"fscorea" - $"fscoreb")
finalscoresDF = finalscoresDF.withColumn("fscoreb-fscorea", $"fscoreb" - $"fscorea")

// Add a Win/loss column Win = 1, Loss = 0
finalscoresDF = finalscoresDF.withColumn("away-win", ($"fscorea-fscoreb" > 0).cast("double"))
finalscoresDF = finalscoresDF.withColumn("home-win", ($"fscoreb-fscorea" > 0).cast("double"))

///////////////////////////////////////////////////////////////////////////////////////////////////////////////
// Create Real time score DF and more wrangling

// Remove Halftime records and these other cases as my datasource doesnt always change the quarter well
// as this particular case isn't handled well... (for now)
var rtscoresDF = rtscoresAndFinalDF.filter(!$"time-string".like("%FINAL%")).
  filter(not($"time-string" === "(HALFTIME)")).
  filter(not($"time-string" === "(12:00 IN 1ST)") ).
  filter(not($"time-string" === "(12:00 IN 2ND)") ).
  filter(not($"time-string" === "(12:00 IN 3RD)") ).
  filter(not($"time-string" === "(12:00 IN 4TH)") ).
  filter(not($"time-string" === "(END OF 1ST)") ).
  filter(not($"time-string" === "(END OF 2ND)") ).
  filter(not($"time-string" === "(END OF 3RD)") ).
  filter(not($"time-string" === "(END OF 4TH)") )

// Create real time score difference
rtscoresDF = rtscoresDF.withColumn("scorea-scoreb", $"scorea" - $"scoreb")
rtscoresDF = rtscoresDF.withColumn("scoreb-scorea", $"scoreb" - $"scorea")

// Create a game PCT complete and PCT left indictor
rtscoresDF = rtscoresDF.withColumn("pct-complete", ((($"timeleft" * -1) + 48 )/48.0)*100)
rtscoresDF = rtscoresDF.withColumn("pct-left", lit(100) - $"pct-complete" )

// Create a unique feature based on my custom UDF.  Idea here is that I have intuition that timeleft and score difference are a strong predictor when combined
rtscoresDF = rtscoresDF.withColumn("cf1", scoredivtimeUdf($"scoreb-scorea", $"pct-left", lit(25.0), lit(0.5)))
rtscoresDF = rtscoresDF.withColumn("cf2", scoredivtimeUdf($"scoreb-scorea", $"pct-left", lit(2.0), lit(1.3)))


###Custom Feature Explanation
After building my initial model, I noticed that the logistic model was adjusting the probabilities well at the end of the games. I had some examples where I had 0 time left in the game, and yet the logistic model was giving a 70% chance of victory for a team. I speculated this was due to the fact that my original features were not fitting the end of game very well. To fix this, I created a spreader custom feature that basically takes the score difference and amplifies it as the score nears the end of the game. This way this feature is very predictive at the end of games and can help adjust the probablities to be more certain at the end of games.

### TODO - Insert Graphs ...

###Lets Take A Look Of What We Have For The Two Dataframes We Just Wrangled

In [39]:
// Some Printouts .....
println("final scores data frame")
finalscoresDF.show(5)
println("Total Games = " + finalscoresDF.count())
println("real time scores data frame")
rtscoresDF.show(5)
println("Total Number of rt score records = " + rtscoresDF.count())


final scores data frame
+----------+--------+-----------+-------+------------+-------+-----------+--------+---------+-----+-----+-------------+----------+------------------+---------------+---------------+--------+--------+
|  dateOrig|      ts|  teamlonga|fscorea|   teamlongb|fscoreb|time-string|timeleft|   gameid|teama|teamb|scorea-scoreb|      date|               key|fscorea-fscoreb|fscoreb-fscorea|away-win|home-win|
+----------+--------+-----------+-------+------------+-------+-----------+--------+---------+-----+-----+-------------+----------+------------------+---------------+---------------+--------+--------+
|2016-04-05|21:22:09|New Orleans|     93|Philadelphia|    107|    (FINAL)|     0.0|400829041|  nor|  phi|          -14|2016-04-05|2016-04-05.nor.phi|            -14|             14|     0.0|     1.0|
|2016-04-05|22:08:42|  Charlotte|     90|     Toronto|     96|    (FINAL)|     0.0|400829043|  cha|  tor|           -6|2016-04-05|2016-04-05.cha.tor|             -6|           

###Inspect Raw Odds Data

In [40]:
println(nbafile)
import sys.process._

// cd /resources/data/
"rm -rf  nbaodds_042516.xml".!
"echo Daily Odds Info".!
"wget https://raw.githubusercontent.com/dustinvanstee/nba-rt-prediction/master/nbaodds_042516.xml".!
"grep title nbaodds_042516.xml"

"echo Historical Odds Info".!
"head -n 5 nbaodds_042516.xml".!


swift://notebooks.spark/scores_nba.test.dat
Daily Odds Info
--2016-09-13 12:16:45--  https://raw.githubusercontent.com/dustinvanstee/nba-rt-prediction/master/nbaodds_042516.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21937 (21K) [text/plain]
Saving to: 'nbaodds_042516.xml'

     0K .......... .......... .                               100% 51.3M=0s

2016-09-13 12:16:45 (51.3 MB/s) - 'nbaodds_042516.xml' saved [21937/21937]

Historical Odds Info


0

basketball_nba.040516.xml:        <title>New Orleans 2.5 O (207.0) 125.0 | Phila. -2.5 U (207.0) -145.0 (Apr 05, 2016 07:10 PM)</title>
basketball_nba.040516.xml:        <title>Detroit 4.0 O (202.0) 160.0 | Miami -4.0 U (202.0) -190.0 (Apr 05, 2016 08:05 PM)</title>
basketball_nba.040516.xml:        <title>Charlotte 4.0 O (200.5) 155.0 | Toronto -4.0 U (200.5) -175.0 (Apr 05, 2016 07:40 PM)</title>
basketball_nba.040516.xml:        <title>Phoenix 14.5 O (207.5) -110.0 | Atlanta -14.5 U (207.5) -110.0 (Apr 05, 2016 08:10 PM)</title>
basketball_nba.040516.xml:        <title>Chicago -3.0 O (201.5) -150.0 | Memphis 3.0 U (201.5) 130.0 (Apr 05, 2016 08:10 PM)</title>


###How to Read the Raw Odds Data
How to interpret the odds data ...
    
    Example Golden State -12.5 O (207.0) -125.0 | Detroit 12.5 U (207.0) 145.0
    The away team is listed first, and the home team is second
    Here Golden State is a 12.5 pt favorite to win.  The over under is in parentheses (207) and is the 50/50 line between teams sum of scores
    being above/below that line.  
    Finally the -125 / +145 numbers are whats known at the moneyline odds. 
        A negative number means you need to bet 125$ to get a 100$ payout
        A positive number means you need to bet 100$ to get a 145$ payout

###Load in Odds Data

In [48]:
///////////////////////////////////////////////////////////////////////////////////////////////////////////////
// Here, the data is very raw, and needs to be pre-processed .  I will start by loading it as an RDD and perform a 
// lot of transformations.  Once I have it properly parsed, I will convert to a dataframe.
// This is not beautiful, but gets the job done
// Data format .....
//       <title>New Orleans 2.5 O (207.0) 125.0 | Phila. -2.5 U (207.0) -145.0 (Apr 05, 2016 07:10 PM)</title>
//       <title>Detroit 4.0 O (202.0) 160.0 | Miami -4.0 U (202.0) -190.0 (Apr 05, 2016 08:05 PM)</title>

// Reading the data in as an RDD first.  There isn't a dataframe parser for this XML I have, so I will write a custom parser ....
val oddsrdd = sc.textFile(oddsfile)

// just grabbing the text within the < ... > tags.  I can do this, because the format is super simple and not nested
val gameStringRdd = oddsrdd.map(x => x.substring(x.indexOf('>')+1,x.lastIndexOf('<')))

// String to Double converter helper
def parseDouble(s: String) = try { Some(s.toDouble) } catch { case _ : Throwable => None }

// This is where I do the heavy lifting of parsing my XML .. and then finally convert my RDD to a dataframe .....
// just lots of string parsing and data type conversions
val oddsDF = gameStringRdd.map(x => {
  // find the period, and then find the space prior ot the period
  // Philadelphia special case.  since later on I index on a decimal point, I am assuming its part of a number and not a team abbrev
  // also removing commas with the cryptic filterNot section ...  
   val x1 = x.replace("Phila.", "Philadelphia").filterNot("," contains _)
   val ss1 = x1.substring(0,x1.indexOf('.'))
   val teamlonga = ss1.substring(0,ss1.lastIndexOf(' '))
   val teama = teamMap(teamlonga)
   val ss2 = x1.substring(ss1.lastIndexOf(' ')+1,x1.length)
   val teamaspread = parseDouble(ss2.substring(0,ss2.indexOf(' ')) )
   val ss3 = ss2.substring(ss2.indexOf(' ')+1,ss2.length)
   val overunder = parseDouble(ss3.substring(ss3.indexOf('(')+1,ss3.indexOf(')')))
   val ss4 = ss3.substring(ss3.indexOf(')')+2,ss3.length)
   val teamaml = parseDouble(ss4.substring(0,ss4.indexOf(' ')))
   val ss5x = ss4.substring(ss4.indexOf('|')+2,ss4.length)
   val ss5 = ss5x.substring(0,ss5x.indexOf('.'))
   
   val teamlongb = ss5.substring(0,ss5.lastIndexOf(' '))
   val teamb = teamMap(teamlongb)
   val ss6 = ss5x.substring(ss5.lastIndexOf(' ')+1,ss5x.length)

   val teambml = parseDouble(ss6.substring(ss6.indexOf(')')+2,ss6.lastIndexOf('(')-1))
   val ss7 = ss6.substring(ss6.lastIndexOf('(')+1,ss6.length)
   val dateInfo = ss7.split(' ')
   val dateStr = dateInfo(2) + "-" + monthMap(dateInfo(0)) + "-" + dateInfo(1)
   // This will become my join key for the other data sets
   val key = dateStr +"." + teama + "." + teamb
  (key,teamlonga,teama,teamaspread,overunder,teamaml,teamlongb,teamb,teambml,dateStr)
   
  }
  // Had to add the groupby and average below because I was getting the game odds over multiple days, and it was 
  // adding noise to the analysis
  // Rename some columns
  // Create a few new columns for later analysis

).toDF("key","teamlonga","teama","teamaspread","overunder","teamaml","teamlongb","teamb","teambml","dateStr").
    groupBy("key","teamlonga","teama","teamlongb","teamb","dateStr").
    agg(avg("teamaspread"),avg("overunder"),avg("teamaml"),avg("teambml")).
    withColumnRenamed("avg(teamaspread)","teamaspread").
    withColumnRenamed("avg(overunder)","overunder").
    withColumnRenamed("avg(teamaml)","teamaml").
    withColumnRenamed("avg(teambml)","teambml").
    withColumn("teambspread", $"teamaspread" * -1).
    withColumn("teama_vegas_fscore", $"overunder" / 2.0 - $"teamaspread"/2.0).
    withColumn("teamb_vegas_fscore", $"overunder" / 2.0 + $"teamaspread"/2.0)

oddsDF.registerTempTable("oddsDF")


###Inspect Some Of The Odds Data

In [50]:
oddsDF.show(5)
printf("Total Home Teams      = %2d\n",oddsDF.select("teama").distinct.sort("teama").count())
printf("Total Away Teams      = %2d\n",oddsDF.select("teamb").distinct.sort("teamb").count())
printf("Total Games Collected = %d\n ",oddsDF.count())


+------------------+-------------+-----+-----------+-----+----------+-----------+---------+-------+-------+-----------+------------------+------------------+
|               key|    teamlonga|teama|  teamlongb|teamb|   dateStr|teamaspread|overunder|teamaml|teambml|teambspread|teama_vegas_fscore|teamb_vegas_fscore|
+------------------+-------------+-----+-----------+-----+----------+-----------+---------+-------+-------+-----------+------------------+------------------+
|2016-04-12.mem.lac|      Memphis|  mem|LA Clippers|  lac|2016-04-12|        7.5|    197.0|  280.0| -340.0|       -7.5|             94.75|            102.25|
|2016-04-06.det.orl|      Detroit|  det|    Orlando|  orl|2016-04-06|       1.25|   209.25|    7.5| -127.5|      -1.25|             104.0|            105.25|
|2016-04-13.uta.lal|         Utah|  uta|  LA Lakers|  lal|2016-04-13|       -5.0|    193.5| -210.0|  175.0|        5.0|             99.25|             94.25|
|2016-04-09.okc.sac|Oklahoma City|  okc| Sacramento|

### Insert Graphs Here

###Join The Odds And Final Score Data Sets

In [52]:
// Here is where we join the Odds/Realtime scores/ Final Scores into one wholistic data set as input for Logistic Machine Learning

// Create a smaller Final Score Dataframe.  Just keep the key, final score a and b, the win/loss indicator
val finalslicedscoresDF = finalscoresDF.select($"key",$"fscorea",$"fscoreb",$"fscorea-fscoreb",$"fscoreb-fscorea",$"away-win",$"home-win")

// First Join the 2 smallest data frames ... odd and final.
val gameDF = oddsDF.join(finalslicedscoresDF, oddsDF("key") === finalscoresDF("key")).drop(oddsDF("key")).
drop("teamlonga").
drop("teamlongb").
drop("teama").
drop("teamb")

// Print Out the Game Dataframe ... notice we have the odds data merged with the win loss data ....
println("gameDF")
gameDF.show(3)
printf("Total Joined Games Collected = %d\n ",gameDF.count())
gameDF.registerTempTable("gameDF")


gameDF
+----------+-----------+---------+-------+-------+-----------+------------------+------------------+------------------+-------+-------+---------------+---------------+--------+--------+
|   dateStr|teamaspread|overunder|teamaml|teambml|teambspread|teama_vegas_fscore|teamb_vegas_fscore|               key|fscorea|fscoreb|fscorea-fscoreb|fscoreb-fscorea|away-win|home-win|
+----------+-----------+---------+-------+-------+-----------+------------------+------------------+------------------+-------+-------+---------------+---------------+--------+--------+
|2016-04-12|        7.5|    197.0|  280.0| -340.0|       -7.5|             94.75|            102.25|2016-04-12.mem.lac|     84|    110|            -26|             26|     0.0|     1.0|
|2016-04-06|       1.25|   209.25|    7.5| -127.5|      -1.25|             104.0|            105.25|2016-04-06.det.orl|    108|    104|              4|             -4|     1.0|     0.0|
|2016-04-13|       -5.0|    193.5| -210.0|  175.0|        5.0| 

### Insert graphs here

###Join The Game Dataframe With The Real Time Score Dataframe

In [54]:
// This is the bigger merge.  Merging the odds/final score data with the real time indicators ...

var lrDF = rtscoresDF.join(gameDF, rtscoresDF("key") === gameDF("key")).drop(gameDF("key"))

println("lrDF : Logistic Regression Data Frame")
lrDF.show(3)
printf("Total Data Points in DataSet = %d\n ",lrDF.count())
lrDF.registerTempTable("lrDF")

lrDF : Logistic Regression Data Frame
+----------+--------+---------+------+-----------+------+--------------+-------------+---------+-----+-----+-------------+----------+-------------+-------------------+-----------------+-------------------+--------------------+----------+-----------+---------+-------+-------+-----------+------------------+------------------+------------------+-------+-------+---------------+---------------+--------+--------+
|  dateOrig|      ts|teamlonga|scorea|  teamlongb|scoreb|   time-string|     timeleft|   gameid|teama|teamb|scorea-scoreb|      date|scoreb-scorea|       pct-complete|         pct-left|                cf1|                 cf2|   dateStr|teamaspread|overunder|teamaml|teambml|teambspread|teama_vegas_fscore|teamb_vegas_fscore|               key|fscorea|fscoreb|fscorea-fscoreb|fscoreb-fscorea|away-win|home-win|
+----------+--------+---------+------+-----------+------+--------------+-------------+---------+-----+-----+-------------+----------+-------

###Add a Few More Features

In [56]:
///////////////////////////////////////////////
// Add an overunder/spread adjusted projection 
// as points are scored during the game
// I found this is a strong indicator
lrDF = lrDF.withColumn("teama_adj_fscore",(( $"pct-complete" * -1)/100 + 1) *  $"teama_vegas_fscore" + $"scorea").
            withColumn("teamb_adj_fscore",(( $"pct-complete" * -1)/100 + 1) *  $"teamb_vegas_fscore" + $"scoreb").
            withColumn("pfscoreb-pfscorea", $"teamb_adj_fscore" - $"teama_adj_fscore" )
                   

###Filter Out Some Data Due To Data Quality

In [60]:
// There is an issue with the data I had captured.  When a quarter transitions from 1st->2nd (etc,etc), sometime the timestring doesn't get updated properly.  Since I used the timestring to calculate the timeleft in the game, I would get some rogue data points.  
// Example, after 1 min in a game, something the two teams would have scores in the 20's, because it was really at 11 mins in the second quarter.  
// My solution was to use the final score sum, and then just scale that down to the time left in the game.  I would then compare to the sum of scores i had, and if it was significantly higher, I would remove them.  I did this by visual inspection ... 
// dfa = departure_from_avg
lrDF = lrDF.withColumn("dfa",(($"fscorea" + $"fscoreb")/48) * ($"timeleft" * -1 + 48) -( $"scorea" + $"scoreb")).orderBy(asc("dfa")).filter($"dfa" > -30)


###Lets Look At Some Stats From Logistic Regression Dataframe

In [61]:
 lrDF.describe().show()

+-------+------------------+------------------+------------------+-------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+-----------------+------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|            scorea|            scoreb|          timeleft|             gameid|     scorea-scoreb|    scoreb-scorea|      pct-complete|          pct-left|               cf1|                cf2|       teamaspread|        overunder|           teamaml|            teambml|        teambspread|teama_vegas_fscore|teamb_vegas_fscore|           fscorea|           fscoreb|   fscorea-fscoreb|   fscoreb-fscorea|          away-win|          home-win|  teama_adj_fscore|  teamb

### Visualize Time Series Data - TBD

### Save Out Dataframe For Further Analysis with Logistic and Linear Regression Notebooks

In [64]:
// Wanted to save out the dataset at this point as I will branch into seperate work efforts for a Logistic/Linear model building
// drop some columns as we move on to next step !!  

lrDF.drop("dateOrig").
    drop("ts").
    drop("teamlonga").
    drop("teamlongb").
    drop("time-string").
    drop("gameid").
    drop("date").
    drop("dateStr").
    drop("teamaml").
    drop("teambml").
    drop("dfa").
    write.format("com.databricks.spark.csv").
    option("header", "true"). // Use first zline of all files as header
    option("inferSchema", "false"). // Automatically infer data types)
    option("nullValue", "empty").
    option("dateFormat", "yyyy-MM-dd").
    option("mode","DROPMALFORMED").
    save("nba-datawrangle-lrDF.csv")


In [67]:
// TODO ## cat a file in swift ?? Prob not possible
val oddsfile = "swift://notebooks.spark/nba-datawrangle-lrDF.csv"

lrDF.drop("dateOrig").
    drop("ts").
    drop("teamlonga").
    drop("teamlongb").
    drop("time-string").
    drop("gameid").
    drop("date").
    drop("dateStr").
    drop("teamaml").
    drop("teambml").
    drop("dfa").
    write.format("com.databricks.spark.csv").
    option("header", "true"). // Use first zline of all files as header
    option("inferSchema", "false"). // Automatically infer data types)
    option("nullValue", "empty").
    option("dateFormat", "yyyy-MM-dd").
    option("mode","DROPMALFORMED").
    save(oddsfile)

